In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import seaborn as sns
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
data = pd.read_csv('../datasets/washing_machine.csv')
print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 48 columns):
ApplStandard                790 non-null object
Brand                       790 non-null object
Cap                         790 non-null float64
CEC Cold                    273 non-null float64
CEC_                        790 non-null int64
Cold Prog                   253 non-null object
Cold Wat Cons               210 non-null float64
Combination                 783 non-null object
Conn_Mode                   790 non-null object
Country                     790 non-null object
delayStartMode              782 non-null object
Depth                       415 non-null float64
DetergentType               783 non-null object
Height                      415 non-null float64
Hot Wat Cons                210 non-null float64
internal_heater             782 non-null object
Loading                     783 non-null object
MachineAction               292 non-null object
Model No                  

In [3]:
# Let us first find see the features which we might have to drop
print(data.columns.values, '\n')

# As we can see here, Grand Date and Old Star Rating are entirely empty so we can drop them
print(data.isnull().sum(), '\n')

# For each of these features, more than 97% of the values are the same, we can drop these as well
print(pd.value_counts(data['N-Standard']), '\n')
print(pd.value_counts(data['Product Class']), '\n')
print(pd.value_counts(data['SubmitStatus']), '\n')
print(pd.value_counts(data['Availability Status']), '\n')

# Since Cold Prog is barely informative without a proper convention, we can drop it,
# Since Hot Wat Cons and Cold Water (L) has more than 70% values missing, we drop that as well
columns = ['Cold Prog', 'Family Name', 'MachineAction', 'Hot Wat Cons', 'Cold Water (L)', 'N-Standard', 'Sold_in', 'Submit_ID', 'GrandDate', 'SubmitStatus', 'Product Class', 'Product Website', 'Star Rating (old)', 'Star Image Large', 'Star Image Small', 'Availability Status', 'Representative Brand URL']
data.drop(columns, axis=1, inplace=True)

['ApplStandard' 'Brand' 'Cap' 'CEC Cold' 'CEC_' 'Cold Prog'
 'Cold Wat Cons' 'Combination' 'Conn_Mode' 'Country' 'delayStartMode'
 'Depth' 'DetergentType' 'Height' 'Hot Wat Cons' 'internal_heater'
 'Loading' 'MachineAction' 'Model No' 'Family Name' 'N-Standard' 'New SRI'
 'New Star' 'postProgenergy' 'powerConsMode' 'Prog Name' 'Sold_in'
 'standbyPowerUsage' 'Submit_ID' 'SubmitStatus' 'Test Prog Time'
 'Tot Wat Cons' 'Type' 'WEI' 'Width' 'ExpDate' 'GrandDate' 'Product Class'
 'Availability Status' 'Product Website' 'Representative Brand URL'
 'Program Time' 'Hot Water (L)' 'Cold Water (L)' 'Star Rating (old)'
 'Star Image Large' 'Star Image Small' 'Registration Number'] 

ApplStandard                  0
Brand                         0
Cap                           0
CEC Cold                    517
CEC_                          0
Cold Prog                   537
Cold Wat Cons               580
Combination                   7
Conn_Mode                     0
Country                       0


In [4]:
print(data.info())
data.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 31 columns):
ApplStandard           790 non-null object
Brand                  790 non-null object
Cap                    790 non-null float64
CEC Cold               273 non-null float64
CEC_                   790 non-null int64
Cold Wat Cons          210 non-null float64
Combination            783 non-null object
Conn_Mode              790 non-null object
Country                790 non-null object
delayStartMode         782 non-null object
Depth                  415 non-null float64
DetergentType          783 non-null object
Height                 415 non-null float64
internal_heater        782 non-null object
Loading                783 non-null object
Model No               790 non-null object
New SRI                790 non-null float64
New Star               790 non-null float64
postProgenergy         782 non-null float64
powerConsMode          782 non-null float64
Prog Name              789 

,ApplStandard,Brand,Cap,CEC Cold,CEC_,Cold Wat Cons,Combination,Conn_Mode,Country,delayStartMode,...,standbyPowerUsage,Test Prog Time,Tot Wat Cons,Type,WEI,Width,ExpDate,Program Time,Hot Water (L),Registration Number
0,AS/NZS 2040.2:2000 (Legacy),Haier,5.0,90.0,480,107.47,NaN,Dual,China,NaN,...,NaN,60.0,128.17,Non-Drum,0.838,NaN,NaN,60.0,136.0,ZCW0001
1,AS/NZS 2040.2:2000 (Legacy),INDESIT,5.5,230.0,240,75.00,NaN,Dual,Italy,NaN,...,NaN,91.0,75.00,Drum,0.720,NaN,NaN,90.0,76.0,ZCW0002
2,AS/NZS 2040.2:2000 (Legacy),INDESIT,6.0,266.0,277,54.00,NaN,Dual,Italy,NaN,...,NaN,145.0,55.00,Drum,0.580,NaN,NaN,145.0,55.0,ZCW0003
3,AS/NZS 2040.2:2000 (Legacy),INDESIT,6.0,133.0,327,57.00,NaN,Dual,Italy,NaN,...,NaN,122.0,68.00,Drum,0.600,NaN,NaN,120.0,72.0,ZCW0004


In [5]:
# Fill up missing value columns
missing_values_columns = [column for column in data.columns.values if data[column].isnull().sum()>0]
print(missing_values_columns)

['CEC Cold', 'Cold Wat Cons', 'Combination', 'delayStartMode', 'Depth', 'DetergentType', 'Height', 'internal_heater', 'Loading', 'postProgenergy', 'powerConsMode', 'Prog Name', 'standbyPowerUsage', 'Width', 'ExpDate']


In [6]:
# Firstly we shall combine Brand and Country into a single column
Brand_Country = []
for index, row in data.iterrows():
    if len(row['Country'])>1: 
        Brand_Country.append((row['Brand'][:2] + row['Country'][0][:1] + row['Country'][1][:1]).upper())
    else: 
        Brand_Country.append((row['Brand'][:2] + row['Country'][0][:1]).upper())
data['Brand_Country'] = Brand_Country
data['Brand_Country']

0      HACH
1      INIT
2      INIT
3      INIT
4      SMCH
5      SMCH
6      MAIT
7      CAIT
8      ASSW
9      ASSW
10     ASSW
11     ASSW
12     WHIN
13     WHIN
14     WHIN
15     WHIN
16     EUCH
17     BRCH
18     BETU
19     EUCH
20     ARIT
21     ARIT
22     ARIT
23     INIT
24     INIT
25     INIT
26     EUCH
27     PATA
28     PATA
29     PATA
       ... 
760    HECH
761    KOCH
762    KOCH
763    KOCH
764    KOCH
765    OMCH
766    HACH
767    LGVI
768    LGVI
769    LGVI
770    LGVI
771    LGVI
772    LGVI
773    LGVI
774    HACH
775    LGVI
776    PACH
777    SMCH
778    ARPO
779    MIGE
780    MIGE
781    INCH
782    SMCH
783    SMCH
784    EVCH
785    LGVI
786    LGVI
787    LGVI
788    LGVI
789    LECH
Name: Brand_Country, Length: 790, dtype: object

In [7]:
mode_cec_cold_overall = data['CEC Cold'].value_counts().index[1]
print('mode of cec_cold - ', mode_cec_cold_overall)
mean = data['CEC Cold'].mean()
std = data['CEC Cold'].std()
null_count = data["CEC Cold"].isnull().sum()
null_random = np.random.randint(mean - std, mean + std, size=null_count)
data['CEC Cold'][np.isnan(data['CEC Cold'])] = null_random
data['CEC Cold'] = data['CEC Cold'].astype(int)
print(data['CEC Cold'].value_counts())
for index, row in data.iterrows():
    cec_cold = data.at[index, 'CEC Cold']
    if cec_cold>=0 and cec_cold<75: value = 0
    elif cec_cold>=75 and cec_cold<125: value = 1
    elif cec_cold>=125 and cec_cold<175: value = 2
    elif cec_cold>=175: value = 3
    data.at[index, 'CEC Cold'] = value
    
data['CEC Cold'] = data['CEC Cold'].astype(int)
print(data['CEC Cold'].value_counts())

mode of cec_cold -  150.0
0      47
112    11
39      9
102     9
150     9
195     8
58      8
70      8
75      8
86      8
87      8
106     8
127     8
84      7
215     7
80      7
184     7
193     7
153     7
57      7
111     7
169     7
120     6
190     6
64      6
67      6
95      6
170     6
90      6
77      6
       ..
22      1
29      1
144     1
52      1
235     1
147     1
230     1
131     1
154     1
129     1
700     1
156     1
123     1
121     1
220     1
113     1
159     1
61      1
103     1
266     1
98      1
177     1
178     1
274     1
143     1
79      1
282     1
289     1
63      1
876     1
Name: CEC Cold, Length: 207, dtype: int64
0    223
1    208
2    180
3    179
Name: CEC Cold, dtype: int64


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [8]:
cold_water_cons_mode = data['Cold Wat Cons'].value_counts().index[1]
print('mode of cold_wat_cons - ', cold_water_cons_mode)
mean = data['Cold Wat Cons'].mean()
std = data['Cold Wat Cons'].std()
null_count = data['Cold Wat Cons'].isnull().sum()
null_random = np.random.randint(mean - std, mean + std, size=null_count)
data['Cold Wat Cons'][np.isnan(data['Cold Wat Cons'])] = null_random
data['Cold Wat Cons'] = data['Cold Wat Cons'].astype(int)
for index, row in data.iterrows():
    cold_wat_cons = data.at[index, 'Cold Wat Cons']
    if cold_wat_cons>=0 and cold_wat_cons<50: value = 0
    elif cold_wat_cons>=50 and cold_wat_cons<80: value = 1
    elif cold_wat_cons>=80: value = 2
    data.at[index, 'Cold Wat Cons'] = value
    
print(data['Cold Wat Cons'].value_counts())

mode of cold_wat_cons -  60.0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


1    292
2    289
0    209
Name: Cold Wat Cons, dtype: int64


In [9]:
combination_mode = data.Combination.value_counts().index[0]
print('mode of combination - ', combination_mode)
data.Combination = data.Combination.fillna(0)
for index, row in data.iterrows():
    if not data.at[index, 'Combination']:
        data.at[index, 'Combination'] = 0
    else: 
        data.at[index, 'Combination'] = 1

data['Combination'] = data['Combination'].astype(int)
data['Combination'].value_counts()

mode of combination -  False


0    702
1     88
Name: Combination, dtype: int64

In [10]:
delayStart_mode = data.Combination.value_counts().index[0]
print('mode of delayStartMode - ', delayStart_mode)
data.delayStartMode = data.delayStartMode.fillna(0)
for index, row in data.iterrows():
    if not data.at[index, 'delayStartMode']:
        data.at[index, 'delayStartMode'] = 0
    else: 
        data.at[index, 'delayStartMode'] = 1

data['delayStartMode'] = data['delayStartMode'].astype(int)
data['delayStartMode'].value_counts()

mode of delayStartMode -  0


1    704
0     86
Name: delayStartMode, dtype: int64

In [11]:
depth_mode = data['Depth'].value_counts().index[0]
print(data.Depth.value_counts())
data.Depth = data.Depth.fillna(0)
for index, row in data.iterrows():
    if int(data.at[index, 'Depth']) == 0:
        brand_country_mode = int(data.loc[data['Brand_Country']==row['Brand_Country']]['Depth'].mode().values[0])
        country_mode = int(data.loc[data['Country']==row['Country']]['Depth'].mode().values[0])
        if brand_country_mode!=0:
            depth = brand_country_mode
        elif country_mode!=0:
            depth = country_mode
        else: 
            depth = depth_mode
    else: 
        depth = int(data.at[index, 'Depth'])
    
    if depth>=292 and depth<565: value = 0
    elif depth==565: value = 1
    elif depth>=570 and depth<700: value = 2
    elif depth>=700 and depth<950: value = 3
    else: value = 4
    
    data.at[index, 'Depth'] = value
    
data.Depth = data.Depth.astype(int)
data.Depth.value_counts()

565.0    48
595.0    24
560.0    19
625.0    18
600.0    16
580.0    14
545.0    13
610.0    13
700.0    13
730.0    11
540.0    11
636.0    11
470.0     9
530.0     9
550.0     9
721.0     9
605.0     8
645.0     7
562.0     7
582.0     7
670.0     6
495.0     6
512.0     6
520.0     6
535.0     6
640.0     6
650.0     5
590.0     5
585.0     4
626.0     4
         ..
624.0     3
620.0     2
696.0     2
448.0     2
616.0     2
536.0     2
695.0     2
767.0     2
537.0     2
643.0     2
542.0     2
850.0     2
606.0     2
658.0     2
686.0     2
505.0     1
634.0     1
611.0     1
525.0     1
557.0     1
675.0     1
714.0     1
660.0     1
510.0     1
400.0     1
517.0     1
950.0     1
555.0     1
835.0     1
637.0     1
Name: Depth, Length: 72, dtype: int64


1    248
2    194
4    170
0    133
3     45
Name: Depth, dtype: int64

In [12]:
mode_detergentType = data.DetergentType.value_counts().index[0]
print('mode is -', mode_detergentType)
data.DetergentType = data.DetergentType.fillna(mode_detergentType)
for index, row in data.iterrows():
    if data.at[index, 'DetergentType'] == 'Drum':
        value = 0
    else: value = 1
    data.at[index, 'DetergentType'] = value

data.DetergentType = data.DetergentType.astype(int)
data.DetergentType.value_counts()

mode is - Drum


0    613
1    177
Name: DetergentType, dtype: int64

In [13]:
height_mode = data['Height'].value_counts().index[0]
# print(data.Height.value_counts())
data.Height = data.Height.fillna(0)
for index, row in data.iterrows():
    if int(data.at[index, 'Height']) == 0:
        brand_country_mode = int(data.loc[data['Brand_Country']==row['Brand_Country']]['Height'].mode().values[0])
        country_mode = int(data.loc[data['Country']==row['Country']]['Height'].mode().values[0])
        if brand_country_mode!=0:
            height = brand_country_mode
        elif country_mode!=0:
            height = country_mode
        else: 
            height = height_mode
    else: 
        height = int(data.at[index, 'Height'])
    
    if height>=515 and height<850: value = 0
    elif height==850: value = 2
    else: value = 3
    
    data.at[index, 'Height'] = height
    
data.Height = data.Height.astype(int)
data.Height.value_counts()

850     551
840      35
845      30
940      19
930      12
1015     11
950      10
1130     10
1026      9
1090      7
1000      6
985       6
980       6
825       6
944       5
1020      5
952       5
990       5
625       4
925       4
600       4
565       3
1070      3
1023      3
960       3
885       2
1050      2
1080      2
984       2
515       2
982       2
920       2
945       2
915       2
700       1
957       1
955       1
935       1
1153      1
1008      1
855       1
1010      1
860       1
900       1
Name: Height, dtype: int64

In [14]:
mode_internal_heater = data.internal_heater.value_counts().index[0]
print(data.internal_heater.value_counts())
data.internal_heater.fillna(mode_internal_heater)
for index, rows in data.iterrows():
    if data.at[index, 'internal_heater']=='Yes on the warm wash program only':
        value = 0
    elif data.at[index, 'internal_heater']=='No':
        value = 1
    else: value = 2
    data.at[index, 'internal_heater'] = value

data.internal_heater = data.internal_heater.astype(int)
data.internal_heater.value_counts()

Yes on the warm wash program only              456
No                                             278
Yes on both the warm and cold wash programs     48
Name: internal_heater, dtype: int64


0    456
1    278
2     56
Name: internal_heater, dtype: int64

In [15]:
mode_loading = data.Loading.value_counts().index[0]
data.Loading = data.Loading.fillna(mode_loading)

data.Loading = data.Loading.map({'Front':0, 'Top':1, 'Twin Tub':2, 'dual drum, front load':3}).astype(int)
data.Loading.value_counts()

0    505
1    275
2      9
3      1
Name: Loading, dtype: int64

In [16]:
# postProgenergy_mode = float(data['postProgenergy'].value_counts().index[0])
# # print(data.postProgenergy.value_counts())
# data.postProgenergy = data.postProgenergy.fillna(0.0)
# for index, row in data.iterrows():
#     if int(data.at[index, 'postProgenergy']) == 0:
#         brand_country_mode = float(data.loc[data['Brand_Country']==row['Brand_Country']]['postProgenergy'].mode().values[0])
#         country_mode = float(data.loc[data['Country']==row['Country']]['postProgenergy'].mode().values[0])
#         if brand_country_mode!=0.0:
#             postProgenergy = '%f' % brand_country_mode
#         elif country_mode!=0.0:
#             postProgenergy = '%f' % country_mode
#         else: 
#             postProgenergy = '%f' % postProgenergy_mode
#     else: 
#         postProgenergy = '%f' % data.at[index, 'postProgenergy']
    
# #     if postProgenergy>=515 and postProgenergy<=565: value = 0
# #     elif postProgenergy>565 and postProgenergy<850: value = 1
# #     elif postProgenergy==850: value = 2
# #     else: value = 3
    
#     data.at[index, 'postProgenergy'] = postProgenergy
    
# # data.postProgenergy = data.postProgenergy.astype(float)
# print(data.postProgenergy.value_counts())
# data.loc[data['postProgenergy']==0.0, 'postProgenergy'] = 0
# data.loc[data['postProgenergy']<0.1, 'postProgenergy'] = 1
# data.loc[data['postProgenergy']>=0.1, 'postProgenergy'] = 2
# data.postProgenergy = data.postProgenergy.astype(int)
# data.postProgenergy.value_counts()